In [57]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUDModule import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = 'aacuser'
password = 'SNHU1234'
host = 'nv-desktop-services.apporto.com'
port = 31077
database = 'AAC'
collection = 'animals'

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)
rescue_types = [
    {'label': 'Water Rescue', 'value': 'Water Rescue'},
    {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain Rescue'},
    {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster Rescue'},
    {'label': 'Reset', 'value': 'Reset'}
]

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Center(html.Img(src='data:Grazioso Salvare Logo/png;base64,{}'.format(encoded_image.decode()))),
    #html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('"Project Two - Darren Sheftic"'))),
    html.Hr(),
        html.Hr(),
     # Radio buttons for each type of rescue animal in the shelter
    dcc.RadioItems(
        id='rescue-type-filter',
        options=rescue_types,
        value='Reset',  # Default value
        labelStyle={'display': 'inline-block'}  # Display options in block style
    ),
    # Generate a horizontal line separating our input from our
    # output element
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True}
                                  for i in df.columns],
                         data=df.to_dict('records'),
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable=False,
                         row_selectable="single",
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[],
                         page_action="native",
                         page_current=0,
                         page_size=10
                        ),
 html.Div(className='row', style={'display': 'flex'},
        children=[
            html.Div(id='graph-id', className='col s12 m6'),  # Pie Chart Placeholder
            html.Div(id='map-id', className='col s12 m6'),  # Map Placeholder
        ]
    ),
    html.Br(),

    # This sets up the output element for the dashboard. The
    # purpose of the stlye option is to make sure that the 
    # output will function like a regular text area and accept
    # newline ('\n') characters as line-breaks.
    html.Div(id="query-out", style={'whiteSpace': 'pre-line'}),
    html.Header(html.H2("Alpha Animal Control Center Dashboard by Darren Sheftic", style={'textAlign': 'center'})),
    #TODO: insert unique identifier code here. Please Note: 
    # when you insert another HTML element here, you will need to 
    # add a comma to the previous line.
])
html.Br(),
html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])


#############################################
# Interaction Between Components / Controller
#############################################



# Callback for the radio buttons to filter data based on rescue type
@app.callback(
    Output('datatable-id', 'data'),
    [Input('rescue-type-filter', 'value')]
)
## FIX ME Add code to filter interactive data table with MongoDB queries
def update_dashboard(filter_type):
    # Start by defining dff as None or as a default DataFrame
    dff = pd.DataFrame()

    # Apply filtering based on the filter_type
    if filter_type == 'Reset':
        dff = df.copy()
    elif filter_type == 'Water Rescue':
        dff = df[(df['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])) &
                 (df['sex_upon_outcome'] == 'Intact Female') &
                 (df['age_upon_outcome_in_weeks'] >= 26) & 
                 (df['age_upon_outcome_in_weeks'] <= 156)]
    elif filter_type == 'Mountain or Wilderness Rescue':
        dff = df[(df['breed'].isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'])) &
                 (df['sex_upon_outcome'] == 'Intact Male') &
                 (df['age_upon_outcome_in_weeks'] >= 26) &
                 (df['age_upon_outcome_in_weeks'] <= 156)]
    elif filter_type == 'Disaster or Individual Tracking':
        dff = df[(df['breed'].isin(['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'])) &
                 (df['sex_upon_outcome'] == 'Intact Male') &
                 (df['age_upon_outcome_in_weeks'] >= 20) &
                 (df['age_upon_outcome_in_weeks'] <= 300)]

    # Ensure dff is not empty before trying to return its data
    if not dff.empty:
        return dff.to_dict('records')
    else:
        # Return an empty list or some default data structure if dff is empty
        return []
    
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])


def update_graphs(viewData):
    # If viewData is None (e.g., before any data is passed to it), return an empty state
    if viewData is None:
        return dash.no_update

    # Convert the viewData (which is a dictionary) into a DataFrame
    dff = pd.DataFrame.from_dict(viewData)

    # Check if 'breed' column exists to avoid KeyError
    if 'breed' in dff.columns:
        # Group by 'breed' and count the occurrences, then reset the index to turn it back into a DataFrame
        breed_counts = dff['breed'].value_counts().reset_index()
        breed_counts.columns = ['breed', 'Count']  # Rename columns for clarity
        
        # Create the pie chart using Plotly Express
        fig = px.pie(breed_counts, values='Count', names='breed', title="Distribution of Breeds")
        
        # Return the chart to the 'graph-id' div
        return dcc.Graph(figure=fig)

    # If 'breed' column doesn't exist, return no update
    return dash.no_update
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
    )
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, selected_rows):
    # If there's no data or no row selected, return the default layout of the map
    if not viewData or not selected_rows:
        return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id")
        ])
    
    # When selected_rows has data, proceed with extracting the data point
    dff = pd.DataFrame.from_dict(viewData)
    
    try:
        # Use the first (and should be only) row index from selected_rows
        row = selected_rows[0]
        
        # Extract the data for the selected row
        selected_data = dff.iloc[row]
        
        # Use the extracted data to update the map marker
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[selected_data['latitude'], selected_data['longitude']], children=[
                    dl.Tooltip(selected_data['breed']),
                    dl.Popup([
                        html.H6("Animal Name"),
                        html.P(selected_data['name'])
                    ])
                ])
            ])
        ]
    except IndexError:
        # Return an error or default state if the index is out of range
        return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id")
        ])


app.run_server(debug=True)


Dash app running on http://127.0.0.1:28024/
